##### The following code analyzes the viable_patches_json file. The points of analysis are described below. The primary tool for this analysis is pydriller.

1. Total size of the cloned repos
2. Total number of vulnerability inducing commits (vuln commits) found & not found
3. Average number of months between vuln commit and patch commit (or fix)
4. Average number of commits between the vuln commit & patch commit (or fix)
5. Average number of vuln commits fixed by patch commit (or fix)
6. Percentage of vulns where the vuln commit and fix were made by the same person


##### Sources
- @inbook{PyDriller,
    title = "PyDriller: Python Framework for Mining Software Repositories",
    abstract = "Software repositories contain historical and valuable information about the overall development of software systems. Mining software repositories (MSR) is nowadays considered one of the most interesting growing fields within software engineering. MSR focuses on extracting and analyzing data available in software repositories to uncover interesting, useful, and actionable information about the system. Even though MSR plays an important role in software engineering research, few tools have been created and made public to support developers in extracting information from Git repository. In this paper, we present PyDriller, a Python Framework that eases the process of mining Git. We compare our tool against the state-of-the-art Python Framework GitPython, demonstrating that PyDriller can achieve the same results with, on average, 50% less LOC and significantly lower complexity.URL: https://github.com/ishepard/pydrillerMaterials: https://doi.org/10.5281/zenodo.1327363Pre-print: https://doi.org/10.5281/zenodo.1327411",
    author = "Spadini, Davide and Aniche, Maurício and Bacchelli, Alberto",
    year = "2018",
    doi = "10.1145/3236024.3264598",
    booktitle = "The 26th ACM Joint European Software Engineering Conference and Symposium on the Foundations of Software Engineering (ESEC/FSE)",
    }

##### Author @Trust-Worthy



##### Reading in the results from the patch_vuln_match.json file and processing objects according to JSONL standard

In [8]:
import pandas as pd
import jsonlines    

json_path:str = "../production_ready/patch_vuln_match.jsonl"

data: list[object] = []

with jsonlines.open(json_path) as reader:

    data = [entry for entry in reader]

# Convert the list of dictionaries into a pandas DataFrame
patch_vuln_df = pd.DataFrame(data)


# Define a function to extract the file paths and commits
def extract_vuln_files_commits(vuln_commits):
    if vuln_commits:
        files = list(vuln_commits.keys())
        commits = [commit for commits in vuln_commits.values() for commit in commits]
        return pd.Series([files, commits])
    else:
        return pd.Series([[], []])  # Empty lists if no vuln_commits

# Apply the function to create new columns
patch_vuln_df[['vuln_files', 'vuln_commits']] = patch_vuln_df['vuln_commits'].apply(extract_vuln_files_commits)



print(patch_vuln_df.head())


           cve_id                     repo  \
0   CVE-1999-0199             bminor/glibc   
1   CVE-1999-0731         KDE/kde1-kdebase   
2   CVE-2002-2443                krb5/krb5   
3  CVE-2005-10002  wp-plugins/secure-files   
4  CVE-2005-10003      mikexstudios/xcomic   

                               patch_commit  \
0  2864e767053317538feafa815046fff89e5a16be   
1  04906bd5de2f220bf100b605dad37b4a1d9a91a6   
2  cf1a0c411b2668c57c41e9c4efd15ba17b6b322c   
3  cab025e5fc2bcdad8032d833ebc38e6bd2a13c92   
4  6ed8e3cc336e29f09c7e791863d0559939da98bf   

                                        vuln_commits  \
0  [dc5efe83c0252ad45337ab98eff6c26fdb29b0a9, 27a...   
1                                                 []   
2  [e88f857c3680ea395c0bed6a82862d8ea1177221, 438...   
3         [b1afc063fd49cfb875e1c6f591543ebff6649469]   
4                                                 []   

                                          vuln_files  
0  [elf/dl-load.c, manual/search.texi, misc/sysl

##### This is where the fun begins.... (iykyk)

In [11]:
"""
Obtaining ...

1. Total size of the cloned repos
2. Total number of vulnerability inducing commits (vuln commits) found & not found
3. Average number of months between vuln commit and patch commit (or fix)
4. Average number of commits between the vuln commit & patch commit (or fix)
5. Average number of vuln commits fixed by patch commit (or fix)
6. Percentage of vulns where the vuln commit and fix were made by the same person
"""


'''
import os
import shutil
from pydriller import Repository, Commit

# Calculate repo size
def get_directory_size(path: str, total_size: float):
    for dirpath, _, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size


count = 1
SIZE_OF_ALL_CLONED_REPOS: list[float] = 0 ### size in MB
TOTAL_NUM_MONTHS: int = 0
TOTAL_PATCH_VULN_PAIRS: int = 0
TOTAL_NUM_COMMITS: int = 0

TOTAL_VULNS: int = 0
BY_SAME_PERSON: int = 0
PERCENTAGE_OF_VULN_N_PATCH_BY_SAME_PERSON: float = 0.0

### Averages --> my goal!
AVERAGE_NUM_MONTHS_BETWEEN_VULN_N_PATCH: float = 0.0
AVERAGE_NUM_COMMITS_BETWEEN_VULN_N_PATCH: float = 0.0



### Point 1, 3, 4 , 6
for owner_repo, patch_commit, vuln_commits in zip(
    patch_vuln_df["repo"], 
    patch_vuln_df["patch_commit"], 
    patch_vuln_df["vuln_commits"]
):
    print("Working on iteration --{count}-- of df)

    # Compose remote repo for pydriller
    owner, repo = owner_repo.split("/")
    remote_url: str = f"https://github.com/{ownner}/{repo}.git"

    TOTAL_VULNS += len(vuln_commits)

    commits_to_analyze: list[str] = []
    commits_to_analyze.append(patch_commit)
    commits_to_analyze.extend(vuln_commits)


    temp_repo: Repository = Repository(remote_url, only_commits=commits_to_analyze, order='reverse')

    patch_author_date: datetime = None
    for commit in temp_repo.traverse_commits():
        size_tracked: bool = False
        commit_count: int = 0
        
        vuln_author_date: datetime = None

        if commit_count == 0:
            patch_author_date = commit.author_date
            commit_count +=1
        else: ### reassing the value of vuln_author_date on each iteration after count > 1
            vuln_author_date = commit.author_date
            commit_count += 1
            TOTAL_PATCH_VULN_PAIRS += 1
        ### Calculate difference between patch date and vuln date in months
        difference: datetime = relativedelta(patch_author_date,vuln_author_date)

        # Get the difference in months
        months_difference = difference.years * 12 + difference.months

        TOTAL_NUM_MONTHS += months_difference

        

            

        
        
        
        
        ### Logic for point 1
        if size_tracked == False:
            temp_repo_path = commit.project_path  # Path to the cloned repo
            repo_size: float = get_directory_size(temp_repo_path, SIZE_OF_CLONED_REPOS) / (1024 * 1024)  # Convert to MB
            SIZE_OF_ALL_CLONED_REPOS += repo_size



        ### Logic for point 3


    
        
        
        
        shutil.rmtree(temp_repo_path)
    
    for vuln_commit in temp_repo_vulns.traverse_commits():
        temp_repo_path = vuln_commit.project_path  # Path to the cloned repo
        


        ### Logic for point 3


    
        
        
        
        shutil.rmtree(temp_repo_path)

    count+=1

AVERAGE_NUM_MONTHS_BETWEEN_VULN_N_PATCH = (TOTAL_NUM_MONTHS / TOTAL_PATCH_VULN_PAIRS )
PERCENTAGE_OF_VULN_N_PATCH_BY_SAME_PERSON = (TOTAL_VULNS / BY_SAME_PERSON )

### Point 2,Point 5
TOTAL_PATCH_COMMITS_w_VULN_COMMIT: int = 0
TOTAL_VULN_COMMITS_FOUND: int = 0
TOTAL_NOT_FOUND: int = 0
    

AVERAGE_NUM_OF_VULNS_TO_PATCH: float = (TOTAL_VULN_COMMITS_FOUND / TAL_PATCH_COMMITS_w_VULN_COMMIT)


### Checks
assert(TOTAL_VULN_COMMITS_FOUND == TOTAL_VULNS)
'''

'\nimport os\nimport shutil\nfrom pydriller import Repository, Commit\n\n# Calculate repo size\ndef get_directory_size(path: str, total_size: float):\n    for dirpath, _, filenames in os.walk(path):\n        for f in filenames:\n            fp = os.path.join(dirpath, f)\n            total_size += os.path.getsize(fp)\n    return total_size\n\n\ncount = 1\nSIZE_OF_ALL_CLONED_REPOS: list[float] = 0 ### size in MB\nTOTAL_NUM_MONTHS: int = 0\nTOTAL_PATCH_VULN_PAIRS: int = 0\nTOTAL_NUM_COMMITS: int = 0\n\nTOTAL_VULNS: int = 0\n\n### Averages --> my goal!\nAVERAGE_NUM_MONTHS_BETWEEN_VULN_N_PATCH: float = 0.0\nAVERAGE_NUM_COMMITS_BETWEEN_VULN_N_PATCH: float = 0.0\n\n\n\n### Point 1, 3, 4 , 6\nfor owner_repo, patch_commit, vuln_commits in zip(\n    patch_vuln_df["repo"], \n    patch_vuln_df["patch_commit"], \n    patch_vuln_df["vuln_commits"]\n):\n    print("Working on iteration --{count}-- of df)\n\n    # Compose remote repo for pydriller\n    owner, repo = owner_repo.split("/")\n    remote_ur